In [1]:
import numpy as np
import matplotlib.pyplot as plt
from utils import *


In [2]:
# Discretization parameters
dx = 0.01 # space step
t_end = 5 # end of simulation time 
gamma = 1.4
c = 0.8

In [3]:
 
x_start = 0
x_end = 10+dx

_x = np.arange(x_start, x_end, dx)

delta_x_ = 1 # Interval of the "disturbance"


### Contact Discontinuity

In [4]:
def top_hat(x, x_start, width):
    # Top hat parametrization
    # 2     ___
    #      |   |
    # 1 ___|   |___
    #   0  1   2  3

    if x_start <= x <= x_start + width:
        return 2
    else:
        return 1

In [5]:
# Contact discontinuity through LFx2
# Velocity is zero
u0 = np.zeros_like(_x)
# Pressure is 1
p0 = np.ones_like(_x)
# Top hat initial condition for density
rho0 = np.array([top_hat(x, 1, 1) for x in _x])


U0 = np.array([rho0, u0, p0]).T

In [6]:
from lfx2 import LFx2
U_prop, a_prop, dt_arr = LFx2(U0, dx, c, t_end, gamma)

In [7]:
#Unpack the solution, U_prop is a 3D array [n_step, n_sample, n_var]
rho_sol = U_prop[:,:,0]

u_sol = U_prop[:,:,1]

p_sol = U_prop[:,:,2]


rho_res = Result(_x, rho_sol, c,"Top Hat" )
u_res = Result(_x, u_sol, c,"Top Hat" )
p_res = Result(_x, p_sol, c,"Top Hat" )





In [8]:
%matplotlib qt

rho_res.animate()


In [9]:
%matplotlib qt

u_res.animate()

In [10]:
%matplotlib qt

p_res.animate()

### Expansion

In [21]:
# Ahead contidions
rho_A = 1.4
u_A =  0
p_A = 1

U0_A = np.array([rho_A, u_A, p_A])


# Behind conditions
## Subsonic (Got a negative pressure that is not physical)
# rho_B = 0.585
# u_B = -0.8
# p_B = 0.295

## Transonic
rho_B = 4.48*10**-4
u_B = -4
p_B = 1.28*10**-5

U0_B = np.array([rho_B, u_B, p_B])

In [22]:
# Take the middle point of the spatial domain
x_middle = x_end/2

# Define the initial condition, with A on the right and B on the right
U0 = np.array([U0_A if x > x_middle else U0_B for x in _x])

# Now we can run the simulation
U_prop, a_prop, dt_arr = LFx2(U0, dx, c, t_end, gamma)

In [23]:
rho_sol = U_prop[:,:,0]
u_sol = U_prop[:,:,1]
p_sol = U_prop[:,:,2]


rho_res = Result(_x, rho_sol, c,"Expansion" )
u_res = Result(_x, u_sol, c,"Expansion"  )
p_res = Result(_x, p_sol, c,"Expansion"  )

In [24]:
%matplotlib qt

rho_res.animate()



In [25]:
%matplotlib qt

u_res.animate()

In [26]:
%matplotlib qt

p_res.animate()



In [27]:
# Add u_sol and a_prop to the plot
UpA = u_sol[:] + np.squeeze(a_prop)
UmA = u_sol[:] - np.squeeze(a_prop)

UpA_res = Result(_x, UpA, c,"Expansion"  )
UmA_res = Result(_x, UmA, c,"Expansion"  )


In [28]:
%matplotlib qt

UpA_res.animate()

In [29]:
%matplotlib qt

UmA_res.animate()

### Compression

### Shock